In [0]:
#IMPORTANT: NEEDS KAGGLE API KEY UPLOADED IN ROOT DIRECTORY

!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

!mkdir data
!kaggle competitions download -p data/ -c modified-mnist

!unzip -d data/ 'data/*.zip'



kaggle.json
 93% 182M/196M [00:01<00:00, 109MB/s] 
100% 196M/196M [00:01<00:00, 107MB/s]
100% 975M/980M [00:07<00:00, 54.4MB/s]
100% 980M/980M [00:07<00:00, 141MB/s] 
  0% 0.00/575k [00:00<?, ?B/s]
100% 575k/575k [00:00<00:00, 58.6MB/s]
Archive:  data/train_max_x.zip
  inflating: data/train_max_x        

Archive:  data/test_max_x.zip
  inflating: data/test_max_x         

2 archives were successfully processed.


In [0]:
!pip install --upgrade tensorflow-gpu


     |████████████████████████████████| 380.8MB 45kB/s 
     |████████████████████████████████| 3.8MB 38.0MB/s 
     |████████████████████████████████| 450kB 46.5MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.1 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.7.0 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found exist

In [0]:

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
%tensorflow_version 2.x

from tensorflow.python.keras.utils import Sequence
import numpy as np

import matplotlib.pyplot as plt

from keras.datasets import mnist
from random import randrange
from keras.utils import to_categorical

#Thruple_Background takes a list of pristine backgrounds as input. 
#And in a "keras.Sequence" fashion will give you batches of pristine 
# background and 3 digits positioned and rotated randomly with degrees 
#between -30 and 30 degrees. Per pristine background __getitem__ generates 
#10 different digit combinations. That value can be modified. 

def make_new_training(x_set, y_set, mnist_train, data_size):
  x_len=len(x_set)
  batch_size=10
  x_train=mnist_train[0]
  y_train=mnist_train[1]

  x_train_range=len(mnist_train[0])
  xdata=list()
  y_data=list()
  
  for i in range(data_size):
    rand1=randrange(0, x_train_range)
    digit1=x_train[rand1]
    result1=y_train[rand1]

    rand2=randrange(0, x_train_range)
    digit2=x_train[rand2]
    result2=y_train[rand2]

    rand3=randrange(0, x_train_range)
    digit3=x_train[rand3]
    result3=y_train[rand3]

    angle1=randrange(-30, 30)
    angle2=randrange(-30, 30)
    angle3=randrange(-30, 30)

    thruple=thruple_mnist(digit1, digit2, digit3, angle1=angle1, angle2=angle2, angle3=angle3)

    #get a pristine box     
    prist_num=randrange(x_len)

    prist_box=pristine_box(x_train[prist_num])

    new_training_sample=combine_pristine_mnist(prist_box[0], thruple)
  
    new_training_sample = np.reshape(new_training_sample, (128,128,1)) #adding fake color channel for CNN

    xdata.append(new_training_sample / 255.) 

    y_data.append(max([result1, result2, result3]))   #have to categorize
    return (np.array(x_data),np.array(y_data) )

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
from keras.datasets import mnist
mnist_train, _ =mnist.load_data()

(ladite, larite)=make_new_training(x_train[:49000], y_train[:49000], mnist_train, 5)

[(6, 10, 25, 19)]
[((0, 0, 6, 128), (0, 19, 128, 128), (25, 0, 128, 128), (0, 0, 128, 10))]


IndexError: ignored

In [0]:
%tensorflow_version 2.x


#from tensorflow.keras.layers import *
#import tensorflow.keras as ks
#import numpy as np

#from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.applications.vgg19 import VGG19
#from tensorflow.keras.activations import relu

model=VGG19(include_top=True, weights=None, input_tensor=None, input_shape=(128, 128, 1), pooling=None, classes=10)




model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True), metrics=["accuracy"])
model.summary()

TensorFlow is already loaded. Please restart the runtime to change versions.
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 1)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
__________________________________________________

In [0]:
from tensorflow.keras.callbacks import History
from tensorflow.keras.models import Model
import pickle
import pandas as pd
import numpy as np
from keras.utils import to_categorical
x_train=pickle.load(open("data/train_max_x", "rb"))
x_train=np.reshape(x_train, (50000, 128, 128, 1))
df = pd.read_csv("data/train_max_y.csv")
y_train = list(df["Label"])
y_train=np.array(y_train)
y_train = to_categorical(y_train)

#keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)




In [0]:
model.fit(x=x_train[:49000], y=y_train[:49000], batch_size=10, epochs=15, validation_data=(x_train[49000:], y_train[49000:]))


Train on 49000 samples, validate on 1000 samples
Epoch 1/15
49000/49000 [==============================] - 937s 19ms/sample - loss: 1.8991 - accuracy: 0.2620 - val_loss: 1.9166 - val_accuracy: 0.2710
Epoch 2/15
49000/49000 [==============================] - 930s 19ms/sample - loss: 1.8863 - accuracy: 0.2673 - val_loss: 1.9068 - val_accuracy: 0.2710
Epoch 3/15
49000/49000 [==============================] - 930s 19ms/sample - loss: 1.8799 - accuracy: 0.2673 - val_loss: 1.9021 - val_accuracy: 0.2710
Epoch 4/15
49000/49000 [==============================] - 930s 19ms/sample - loss: 1.8691 - accuracy: 0.2688 - val_loss: 1.8728 - val_accuracy: 0.2710
Epoch 5/15
49000/49000 [==============================] - 922s 19ms/sample - loss: 1.7622 - accuracy: 0.3171 - val_loss: 1.5845 - val_accuracy: 0.3920
Epoch 6/15
49000/49000 [==============================] - 918s 19ms/sample - loss: 1.4262 - accuracy: 0.4687 - val_loss: 1.4366 - val_accuracy: 0.4630
Epoch 7/15
49000/49000 [=====================

In [0]:

from tensorflow.keras.models import load_model
model = load_model('VGG19model3.h5')

In [0]:
import numpy as np

x_test=np.reshape(pickle.load(open("data/test_max_x", "rb")), (10000, 128, 128, 1))

preds=model.predict(x_test)
preds=[np.argmax(x) for x in preds]

In [0]:
import pandas as pd

df = pd.DataFrame(preds, columns=["label"])
df.to_csv('list.csv', index=False)



In [0]:
model.save("VGG19model4epoch21.h5")

In [0]:
model.fit(x=x_train[:49000], y=y_train[:49000], batch_size=10, epochs=20, validation_data=(x_train[49000:], y_train[49000:]), initial_epoch=19)

Train on 49000 samples, validate on 1000 samples
Epoch 20/20
49000/49000 [==============================] - 909s 19ms/sample - loss: 0.0945 - accuracy: 0.9707 - val_loss: 0.2676 - val_accuracy: 0.9340


In [0]:
model.fit(x=x_train[:49000], y=y_train[:49000], batch_size=10, epochs=21, validation_data=(x_train[49000:], y_train[49000:]), initial_epoch=20)

Train on 49000 samples, validate on 1000 samples
Epoch 21/21
49000/49000 [==============================] - 900s 18ms/sample - loss: 0.0914 - accuracy: 0.9717 - val_loss: 0.2487 - val_accuracy: 0.9280


In [0]:

from keras.utils import to_categorical

new_x_train_final=np.array(new_x_train0+new_x_train1)
new_y_train_final=new_y_train0+new_y_train1
new_y_train_final=np.reshape(np.array(new_x_train_final), (18052, 128, 128, 1))
new_y_train_final = to_categorical(new_y_train_final)

Using TensorFlow backend.


NameError: ignored

In [0]:
model.fit(x=new_x_train_final, y=new_y_train_final, batch_size=10, epochs=23, validation_data=(x_train[49000:], y_train[49000:]), initial_epoch=20)

ValueError: ignored

In [0]:
preds=model.predict(np.reshape(x_train[49000:], (1000, 128,128,1)))

In [0]:
np.argmax(y_train[2])

2

In [0]:
count=0
for i in range(len(preds)):
  if np.argmax(preds[i])==np.argmax(y_train[i]):
    count+=1
print(count/len(preds))

0.188


In [0]:
preds[2]

array([4.7040125e-03, 4.7959760e-02, 1.5949409e-01, 2.4970497e-01,
       5.2116394e-01, 8.3295023e-03, 5.1118969e-04, 5.4886574e-03,
       1.1174501e-03, 1.5263808e-03], dtype=float32)

In [0]:
import numpy as np
from skimage.measure import label, regionprops
from skimage.transform import *

digit_threshold = np.vectorize(lambda x: 255 if x > 220 else 0)
def digit_extractor(source, top_clusters=3):
    source = np.reshape(source, (128,128))

    threshold_img = digit_threshold(source)
    declustered_img = remove_clusters_below_size(threshold_img, size=5)
    dilated_img = dilation(declustered_img)
    labelled_img = label(dilated_img, connectivity = 2)

    nb_clusters = np.max(labelled_img)

    cluster_list = []
    for i in range(1, nb_clusters + 1): #ignoring background
        cluster_size = np.sum(labelled_img == i)
        cluster_list.append((cluster_size, i))  # append tuple

    cluster_list.sort(reverse=True)

    cluster_imgs = []
    for _, cluster in cluster_list[:top_clusters + 1]: #assuming that the background cluster is the largest

        mask = labelled_img == cluster
        unmasked = unmask(source, mask)
        cropped_digit = crop(unmasked, mask)
        padded_digit = center_by_bounding_box(cropped_digit)

        cluster_imgs.append(padded_digit)

    return cluster_imgs

def unmask(source, mask):
    return source * mask

def crop(source, mask):
    width = source.shape[0]
    height = source.shape[1]

    h_presence = np.sum(mask, axis=0)
    v_presence = np.sum(mask, axis=1)

    h_min = min([i for i in range(width) if h_presence[i]])
    h_max = max([i for i in range(width) if h_presence[i]])

    v_min = min([i for i in range(height) if v_presence[i]])
    v_max = max([i for i in range(height) if v_presence[i]])

    return source[v_min:v_max, h_min:h_max]


def center_by_bounding_box(source, box_size=28):
    height = source.shape[0]
    width = source.shape[1]

    cropped_source = source

    if height > box_size:
        excedent = int((height - box_size)/2)
        cropped_source = cropped_source[excedent:excedent+box_size, : ]

        height = box_size

    if width > box_size:
        excedent = int((width - box_size) / 2)
        cropped_source = cropped_source[ : , excedent:excedent + box_size]

        width = box_size

    canvas = np.zeros((box_size, box_size))  # zero array of size box_size

    height_pad = int((box_size - height) / 2)
    width_pad = int((box_size - width) / 2)

    canvas[height_pad:height_pad + height, width_pad:width_pad + width] = cropped_source

    return canvas


def remove_clusters_below_size(img, size=5):
    labelled_img, nb_clusters = label(img, connectivity=2, return_num=True)
    cluster_ids_to_eliminate = [0]  # 0 is the background cluster

    for i in range(1, nb_clusters):
        cluster_size = np.sum(labelled_img == i)

        if size is not None and cluster_size <= size:
            cluster_ids_to_eliminate.append(i)

    elimination_map = lambda x: 0 if x in cluster_ids_to_eliminate else 255
    vem = np.vectorize(elimination_map)
    declustered_img = vem(labelled_img)
    return declustered_img

def threshold(source, threshold=220):
    f = lambda x : 255 if x >= threshold else 0
    vf = np.vectorize(f)

    return vf(source)

#new methods to get a pristine 28x28 box on which to add a digit. Use data_creator.py

def pristine_box(img):#can produce waaaay more

    bounding_boxes=get_bounding_boxes(img)
    if bounding_boxes==None:
      return None
    subdiv=get_subdiv(bounding_boxes)
    legal_box_dim=get_best_intersections(subdiv)
    if legal_box_dim==None:
      return None

    pristine=[np.array(resize(img[box_dim[0]:box_dim[2], box_dim[1]:box_dim[3]], (128,128))) for box_dim in legal_box_dim]

    return pristine

def get_bounding_boxes(img):

    threshold_img=threshold(img)
    labelled_temp, nb_clusters = label(threshold_img, connectivity=2, return_num=True) #could avoid recalling this
    props=regionprops(labelled_temp)
    bounding_boxes=[x.bbox for x in props]
    if len(bounding_boxes)!=3:
      return None
    return bounding_boxes

def get_subdiv(bounding_boxes):
    subdiv=[]
    size=128
    for x in bounding_boxes:
        top=(0, 0, x[0], size)
        right=(0, x[3], size, size)
        bot=(x[2], 0, size, size)
        left=(0, 0, size, x[1])
        subdiv.append((top, right, bot, left))
    
    return subdiv


def get_intersections(subdiv, thresh=28*28): #assuming we have 3 clusters
    legal=[]
    for x in subdiv[0]:
        for y in subdiv[1]:
            for z in subdiv[2]:
                box=intersection(x, y, z)
                size=(box[2]-box[0])*(box[3]-box[1])
                if size>thresh:
                    legal.append(box)
    return legal

def get_best_intersections(subdiv):
    legal=[]
    min_box=(0, 0, 0, 0)
    min_size=1000000000000000000000000000000

    for x in subdiv[0]:
      for y in subdiv[1]:
        for z in subdiv[2]:
          box=intersection(x, y, z)
          size=(box[2]-box[0])*(box[3]-box[1])
          if size<min_size:
            min_box=box
    legal+=[box]

    if min_size==0 or min_box==(0, 0, 0, 0):
      return None

    return legal



def intersection(x, y, z):
    (minrow, mincol, maxrow, maxcol)=(max(x[0], y[0], z[0]), max(x[1], y[1], z[1]), min(x[2], y[2], z[2]), min(x[3], y[3], z[3]))
    if minrow>=maxrow or mincol>=maxcol:
        return (0, 0, 0, 0)
    else:
        return (minrow, mincol, maxrow, maxcol)



def reshape_pristine(pristine):
    (width, height)=pristine.shape

    if width<28 or height<28:
        print("kawaboonga")
        return None
    else:
        w_cush=int((width-28)/2)
        h_cush=int((height-28)/2)
        w2=w_cush+28
        h2=h_cush+28
        return pristine[w_cush:w2, h_cush:h2]

threshold2 = np.vectorize(lambda x: x if x < 255 else 255)

def combine_pristine_mnist(pristine, mnist):
    return threshold2(pristine + mnist)

def thruple_mnist(digit1, digit2, digit3, angle1=0, angle2=0, angle3=0):
    img=np.zeros((128, 128))
    
    #digit1=resize(digit1, (28, 28))
    #digit2=resize(digit2, (28, 28))
    #digit3=resize(digit3, (28, 28))
    
    x_1=randrange(0, 100)
    y_1=randrange(0, 100)

    x_2=randrange(0, 100)
    y_2=randrange(0, 100)
    
    while (x_2<x_1+28 and x_2>=x_1-28) and (y_2<y_1+28 and y_2>=y_1-28):
        x_2=randrange(0, 100)
        y_2=randrange(0, 100)

    x_3=randrange(0, 100)
    y_3=randrange(0, 100)

    while ((x_3<x_1+28 and x_3>=x_1-28) and (y_3<y_1+28 and y_3>=y_1-28) or (x_3<x_2+28 and x_3>=x_2-28) and (y_3<y_2+28 and y_3>=y_2-28)):
        x_3=randrange(0, 100)
        y_3=randrange(0, 100)

    img[x_1:x_1+28, y_1:y_1+28]=rotate(digit1, angle1, preserve_range=True)
    img[x_2:x_2+28, y_2:y_2+28]=rotate(digit2, angle2, preserve_range=True)
    img[x_3:x_3+28, y_3:y_3+28]=rotate(digit3, angle3, preserve_range=True)

    return img

In [0]:
from tensorflow.keras.callbacks import History
from tensorflow.keras.models import Model
import pickle
import pandas as pd
import numpy as np
from keras.utils import to_categorical
from keras.datasets import mnist

(x_mnist, y_mnist), _=mnist.load_data()

x_train=pickle.load(open("data/train_max_x", "rb"))
y_train=pd.read_csv("data/train_max_y.csv")


def generate_data(x_train_max, y_train_max, x_mnist, y_mnist, size):

  len_train_max=len(x_train_max)
  len_mnist=len(x_mnist)

  x_data=[]
  y_data=[]
  for i in range(size):
    if i%4000==0:
      print(i)
    
    rand1=randrange(len_mnist)
    digit1=x_mnist[rand1]
    result1=y_mnist[rand1]

    rand2=randrange(len_mnist)
    digit2=x_mnist[rand2]
    result2=y_mnist[rand2]

    rand3=randrange(len_mnist)
    digit3=x_mnist[rand3]
    result3=y_mnist[rand3]


    angle1=randrange(-30, 30)
    angle2=randrange(-30, 30)
    angle3=randrange(-30, 30)

    thruple=thruple_mnist(digit1, digit2, digit3, angle1=angle1, angle2=angle2, angle3=angle3)

    rand_train_max=randrange(len_train_max)
    

    x=x_train_max[rand_train_max]

    pristine=pristine_box(x)

    if pristine!=None:
      rand_last=randrange(len(pristine))
      new=combine_pristine_mnist(pristine[rand_last], thruple)
      x_data+=[new]
      y_data+=[np.max([result1, result2, result3])]

  return (x_data, y_data)




Using TensorFlow backend.


In [0]:
from random import randrange
new_x_train1, new_y_train1=generate_data(x_train, y_train, x_mnist, y_mnist, 20000)

0


KeyboardInterrupt: ignored

In [0]:
len(new_x_train1)

9031

In [0]:
import pandas as pd
df = pd.DataFrame(new_y_train, columns=["label"])
df.to_csv('new_y_train0.csv', index=False)

In [0]:
pickle.dump(pristine, open("data/pristine_box_"+str(i)+".pkl", "wb"))

In [0]:
lala=pickle.load(open("new_x_train0", "rb"))

EOFError: ignored